## Loan Status Prediction

- Main aim of this dataset to predict which of the customers will have their loan dataset
- Using machine learning model to predict it
- Dataset is Load prediction

In [76]:
##Import Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_selection import RFE


In [27]:
data=pd.read_csv('LoanData.csv',index_col=None)
data.head(5)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


# Define the Function for FEATURE SELECTION/ Model and Evaluation metrics

In [122]:
def selectkbest(indep_X, dep_Y, n, feature_names):
    test = SelectKBest(score_func=chi2, k=n)
    fit1 = test.fit(indep_X, dep_Y)
    selectk_features = fit1.transform(indep_X)
    selected_features = fit1.get_support()
    selected_feature_names = [feature_names[i] for i, selected in enumerate(selected_features) if selected]
    return selectk_features, selected_features, selected_feature_names

def print_metrics(classifier_name, accuracy, cm, classification_report):
    print("=" * 40)
    print(f"{classifier_name}\nAccuracy: {accuracy}\nConfusion Matrix:\n{cm}\nClassification Report:\n{classification_report}\n")

def split_scalar(indep_X, dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test, y_train, y_test

def cm_prediction(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    Accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return classifier, Accuracy, report, X_test, y_test, cm

def logistic(X_train, y_train, X_test, y_test):
    classifier = LogisticRegression(random_state=0)
    classifier.fit(X_train, y_train)
    classifier, Accuracy, report, X_test, y_test, cm = cm_prediction(classifier, X_test, y_test)
    return classifier, Accuracy, report, X_test, y_test, cm

def svm_linear(X_train, y_train, X_test, y_test):
    classifier = SVC(kernel='linear', random_state=0)
    classifier.fit(X_train, y_train)
    classifier, Accuracy, report, X_test, y_test, cm = cm_prediction(classifier, X_test, y_test)
    return classifier, Accuracy, report, X_test, y_test, cm

def svm_NL(X_train, y_train, X_test, y_test):
    classifier = SVC(kernel='rbf', random_state=0)
    classifier.fit(X_train, y_train)
    classifier, Accuracy, report, X_test, y_test, cm = cm_prediction(classifier, X_test, y_test)
    return classifier, Accuracy, report, X_test, y_test, cm

def Navie(X_train, y_train, X_test, y_test):
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)
    classifier, Accuracy, report, X_test, y_test, cm = cm_prediction(classifier, X_test, y_test)
    return classifier, Accuracy, report, X_test, y_test, cm

def knn(X_train, y_train, X_test, y_test):
    classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
    classifier.fit(X_train, y_train)
    classifier, Accuracy, report, X_test, y_test, cm = cm_prediction(classifier, X_test, y_test)
    return classifier, Accuracy, report, X_test, y_test, cm

def Decision(X_train, y_train, X_test, y_test):
    classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    classifier, Accuracy, report, X_test, y_test, cm = cm_prediction(classifier, X_test, y_test)
    return classifier, Accuracy, report, X_test, y_test, cm

def random(X_train, y_train, X_test, y_test):
    classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    classifier, Accuracy, report, X_test, y_test, cm = cm_prediction(classifier, X_test, y_test)
    return classifier, Accuracy, report, X_test, y_test, cm

def selectk_Classification(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf): 
    dataframe = pd.DataFrame(index=['ChiSquare'],
                             columns=['Logistic', 'SVMl', 'SVMnl', 'KNN', 'Navie', 'Decision', 'Random'])
    for number, idex in enumerate(dataframe.index):      
        dataframe['Logistic'][idex] = acclog[number]       
        dataframe['SVMl'][idex] = accsvml[number]
        dataframe['SVMnl'][idex] = accsvmnl[number]
        dataframe['KNN'][idex] = accknn[number]
        dataframe['Navie'][idex] = accnav[number]
        dataframe['Decision'][idex] = accdes[number]
        dataframe['Random'][idex] = accrf[number]
    return dataframe

In [68]:
data

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849.0,0.0,128.0,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583.0,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000.0,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,2900.0,0.0,71.0,360.0,1.0,Rural,Y
610,Male,Yes,3+,Graduate,No,4106.0,0.0,40.0,360.0,1.0,Rural,Y
611,Male,Yes,1,Graduate,No,8072.0,240.0,253.0,360.0,1.0,Urban,Y
612,Male,Yes,2,Graduate,No,7583.0,0.0,187.0,360.0,1.0,Urban,Y


In [69]:
# Convert categorical to numerical one
data=pd.get_dummies(data,drop_first=True)

In [71]:
data.head(3)

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_Y
0,5849.0,0.0,128.0,360.0,1.0,1,0,0,0,0,0,0,0,1,1
1,4583.0,1508.0,128.0,360.0,1.0,1,1,1,0,0,0,0,0,0,0
2,3000.0,0.0,66.0,360.0,1.0,1,1,0,0,0,0,1,0,1,1


In [72]:
indep_X=data.drop('Loan_Status_Y', 1)
dep_Y=data['Loan_Status_Y']

In [73]:
indep_X

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban
0,5849.0,0.0,128.0,360.0,1.0,1,0,0,0,0,0,0,0,1
1,4583.0,1508.0,128.0,360.0,1.0,1,1,1,0,0,0,0,0,0
2,3000.0,0.0,66.0,360.0,1.0,1,1,0,0,0,0,1,0,1
3,2583.0,2358.0,120.0,360.0,1.0,1,1,0,0,0,1,0,0,1
4,6000.0,0.0,141.0,360.0,1.0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,2900.0,0.0,71.0,360.0,1.0,0,0,0,0,0,0,0,0,0
610,4106.0,0.0,40.0,360.0,1.0,1,1,0,0,1,0,0,0,0
611,8072.0,240.0,253.0,360.0,1.0,1,1,1,0,0,0,0,0,1
612,7583.0,0.0,187.0,360.0,1.0,1,1,0,1,0,0,0,0,1


In [74]:
dep_Y

0      1
1      0
2      1
3      1
4      1
      ..
609    1
610    1
611    1
612    1
613    0
Name: Loan_Status_Y, Length: 614, dtype: uint8

In [106]:
data.columns

Index(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Gender_Male', 'Married_Yes',
       'Dependents_1', 'Dependents_2', 'Dependents_3+',
       'Education_Not Graduate', 'Self_Employed_Yes',
       'Property_Area_Semiurban', 'Property_Area_Urban', 'Loan_Status_Y'],
      dtype='object')

In [123]:
feature_names = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Gender_Male', 'Married_Yes',
       'Dependents_1', 'Dependents_2', 'Dependents_3+',
       'Education_Not Graduate', 'Self_Employed_Yes',
       'Property_Area_Semiurban', 'Property_Area_Urban', 'Loan_Status_Y']
kbest, selected_features, selected_feature_names = selectkbest(indep_X, dep_Y, 10, feature_names)

acclog = []
accsvml = []
accsvmnl = []
accknn = []
accnav = []
accdes = []
accrf = []

print("Selected Features:", selected_feature_names)

X_train, X_test, y_train, y_test = split_scalar(kbest, dep_Y)

logistic_classifier, accuracy_log, report_log, X_test, y_test, cm_log = logistic(X_train, y_train, X_test, y_test)
acclog.append(accuracy_log)
print_metrics("Logistic Regression", accuracy_log, cm_log, report_log)


svm_linear_classifier, accuracy_svcl, report_svcl, X_test, y_test, cm_svcl = svm_linear(X_train, y_train, X_test, y_test)
accsvml.append(accuracy_svcl)
print_metrics("SVM Linear", accuracy_svcl, cm_svcl, report_svcl)


svm_NL_classifier, accuracy_svcnl, report_svcnl, X_test, y_test, cm_svcnl = svm_NL(X_train, y_train, X_test, y_test)
accsvmnl.append(accuracy_svcnl)
print_metrics("SVM Non-Linear", accuracy_svcnl, cm_svcnl, report_svcnl)


knn_classifier, accuracy_knn, report_knn, X_test, y_test, cm_knn = knn(X_train, y_train, X_test, y_test)
accknn.append(accuracy_knn)
print_metrics("KNN", accuracy_knn, cm_knn, report_knn)


navie_classifier, accuracy_nav, report_nav, X_test, y_test, cm_nav = Navie(X_train, y_train, X_test, y_test)
accnav.append(accuracy_nav)
print_metrics("Naive Bayes", accuracy_nav, cm_nav, report_nav)


decision_classifier, accuracy_dec, report_dec, X_test, y_test, cm_dec = Decision(X_train, y_train, X_test, y_test)
accdes.append(accuracy_dec)
print_metrics("Decision Tree", accuracy_dec, cm_dec, report_dec)


random_classifier, accuracy_random, report_random, X_test, y_test, cm_random = random(X_train, y_train, X_test, y_test)
accrf.append(accuracy_random)
print_metrics("Random Forest", accuracy_random, cm_random, report_random)



Selected Features: ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Married_Yes', 'Dependents_1', 'Dependents_2', 'Dependents_3+', 'Education_Not Graduate', 'Property_Area_Semiurban', 'Property_Area_Urban']
Logistic Regression
Accuracy: 0.7207792207792207
Confusion Matrix:
[[  3  40]
 [  3 108]]
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.07      0.12        43
           1       0.73      0.97      0.83       111

    accuracy                           0.72       154
   macro avg       0.61      0.52      0.48       154
weighted avg       0.67      0.72      0.64       154


SVM Linear
Accuracy: 0.7207792207792207
Confusion Matrix:
[[  0  43]
 [  0 111]]
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        43
           1       0.72      1.00      0.84       111

    accuracy                           0.72       154
   macro avg       0.36

C:\Users\Arun\anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Arun\anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Arun\anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [116]:
result=selectk_Classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf)

result#10


,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.720779,0.720779,0.714286,0.636364,0.701299,0.584416,0.623377


- Based on the results, it depends on the importance of precision, recall, and accuracy in your specific case. If correctly identifying class 1 (approved loans) is crucial and you can tolerate some false positives, SVM Linear or Non-Linear might be suitable. If you need a balance between classes and are willing to sacrifice some accuracy, Logistic Regression or Naive Bayes could be considered.
- In this case I could tolerate false positive and choose SVM linear model

In [129]:
X_train.shape

(460, 10)

In [130]:
len(selected_features)

14

In [82]:
results = selectk_Classification(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)
results#10

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.720779,0.720779,0.714286,0.636364,0.701299,0.584416,0.623377


In [88]:
import pickle
filename='finaized_model_svc.sav'

In [89]:
pickle.dump(svm_linear_classifier,open(filename,'wb'))

In [134]:
new_data=new_data[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Married', 'Dependents', 'Education', 'Property_Area']]

In [146]:
new_data=new_data.drop('Loan_ID',axis=1)
new_data

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
614,Male,Yes,0,Graduate,No,5720,0.0,110.0,360.0,1.0,Urban,NaN
615,Male,Yes,1,Graduate,No,3076,1500.0,126.0,360.0,1.0,Urban,NaN
616,Male,Yes,2,Graduate,No,5000,1800.0,208.0,360.0,1.0,Urban,NaN
617,Male,Yes,2,Graduate,No,2340,2546.0,100.0,360.0,NaN,Urban,NaN
618,Male,No,0,Not Graduate,No,3276,0.0,78.0,360.0,1.0,Urban,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
976,Male,Yes,3+,Not Graduate,Yes,4009,1777.0,113.0,360.0,1.0,Urban,NaN
977,Male,Yes,0,Graduate,No,4158,709.0,115.0,360.0,1.0,Urban,NaN
978,Male,No,0,Graduate,No,3250,1993.0,126.0,360.0,NaN,Semiurban,NaN
979,Male,Yes,0,Graduate,No,5000,2393.0,158.0,360.0,1.0,Rural,NaN


In [147]:
new_data=pd.get_dummies(new_data,drop_first=True)

In [153]:
new_data1=new_data[['ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Married_Yes',
 'Dependents_1',
 'Dependents_2',
 'Dependents_3+',
 'Education_Not Graduate',
 'Property_Area_Semiurban',
 'Property_Area_Urban']
]

In [151]:
selected_feature_names

['ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Married_Yes',
 'Dependents_1',
 'Dependents_2',
 'Dependents_3+',
 'Education_Not Graduate',
 'Property_Area_Semiurban',
 'Property_Area_Urban']

In [159]:
new_data1.isnull().sum()

ApplicantIncome            0
CoapplicantIncome          0
LoanAmount                 0
Married_Yes                0
Dependents_1               0
Dependents_2               0
Dependents_3+              0
Education_Not Graduate     0
Property_Area_Semiurban    0
Property_Area_Urban        0
dtype: int64

In [158]:
new_data1['LoanAmount'].fillna(new_data1['LoanAmount'].median(), inplace=True)

C:\Users\Arun\anaconda3\envs\aiml\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [160]:
new_data_predictions = svm_linear_classifier.predict(new_data1)

# 'new_data_predictions' now contains the predicted labels for the new data
print("Predicted Labels for New Data:", new_data_predictions)

Predicted Labels for New Data: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


C:\Users\Arun\anaconda3\envs\aiml\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


## Deployment Phase

In [162]:
import pickle
filename='finalized_model_svc_linear.sav'

In [164]:
pickle.dump(svm_linear_classifier,open(filename,'wb'))

In [167]:
new_data1=new_data1[['ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Married_Yes',
 'Dependents_1',
 'Dependents_2',
 'Dependents_3+',
 'Education_Not Graduate',
 'Property_Area_Semiurban',
 'Property_Area_Urban']
]

In [169]:
new_data1.head(1)

,ApplicantIncome,CoapplicantIncome,LoanAmount,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Property_Area_Semiurban,Property_Area_Urban
614,5720,0.0,110.0,1,0,0,0,0,0,1


In [170]:
loaded_model=pickle.load(open(filename,'rb'))
result=loaded_model.predict(new_data1)
result

C:\Users\Arun\anaconda3\envs\aiml\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [176]:
result_mapped = pd.Series(result).map({1: 'Y', 0: 'N'})
predicted_df = pd.DataFrame({'Loan_Status': result_mapped})
predicted_df

,Loan_Status
0,Y
1,Y
2,Y
3,Y
4,Y
...,...
362,Y
363,Y
364,Y
365,Y


In [180]:
new_data1 = new_data1.reset_index(drop=True)


In [181]:
new_data1

,ApplicantIncome,CoapplicantIncome,LoanAmount,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Property_Area_Semiurban,Property_Area_Urban
0,5720,0.0,110.0,1,0,0,0,0,0,1
1,3076,1500.0,126.0,1,1,0,0,0,0,1
2,5000,1800.0,208.0,1,0,1,0,0,0,1
3,2340,2546.0,100.0,1,0,1,0,0,0,1
4,3276,0.0,78.0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
362,4009,1777.0,113.0,1,0,0,1,1,0,1
363,4158,709.0,115.0,1,0,0,0,0,0,1
364,3250,1993.0,126.0,0,0,0,0,0,1,0
365,5000,2393.0,158.0,1,0,0,0,0,0,0


In [187]:
predicted_df = pd.concat([new_data1, pd.DataFrame({'Loan_Status': result_mapped})], axis=1)
predicted_df

,ApplicantIncome,CoapplicantIncome,LoanAmount,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Property_Area_Semiurban,Property_Area_Urban,Loan_Status
0,5720,0.0,110.0,1,0,0,0,0,0,1,Y
1,3076,1500.0,126.0,1,1,0,0,0,0,1,Y
2,5000,1800.0,208.0,1,0,1,0,0,0,1,Y
3,2340,2546.0,100.0,1,0,1,0,0,0,1,Y
4,3276,0.0,78.0,0,0,0,0,1,0,1,Y
...,...,...,...,...,...,...,...,...,...,...,...
362,4009,1777.0,113.0,1,0,0,1,1,0,1,Y
363,4158,709.0,115.0,1,0,0,0,0,0,1,Y
364,3250,1993.0,126.0,0,0,0,0,0,1,0,Y
365,5000,2393.0,158.0,1,0,0,0,0,0,0,Y
